提取数据并输出前一交易日信息

In [1]:
from macrotoolchain.mainAPI import *
from lowrisk.AssetAllocation import * 
import pandas as pd
import numpy as np
import datetime
import pypfopt as opt #加载组合构建包

In [2]:
price = pd.read_excel('price.xlsx',index_col=0) #读取本地保存数据

In [3]:
hs300 = edbData(seriesCode='M0020209', startingDate=Date.fromString(date='20210101',format='%Y%m%d'), title='沪深300指数')
hs300 = DataToDf({'沪深300':hs300}) #为了获取交易日序列
startDate = hs300.index[np.where(hs300.index == price.index[-1])[0][0] - 1].strftime('%Y%m%d') #计算提取数据的开始日期

In [4]:
#获取最新的数据
AL_Close = FutureMainData(which='AL',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
AU_Close = FutureMainData(which='AU',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
CU_Close = FutureMainData(which='CU',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
B_Close = FutureMainData(which='SC',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
PTA_Close = FutureMainData(which='TA',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
CBOT_Close = FutureMainData(which='S.CBT',what='close',startDate=startDate,
                            endDate=Date.today().toString('%Y%m%d'),return_Data=True)
P_Close = FutureMainData(which='P',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
RB_Close = FutureMainData(which='RB',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
IF_Close = FutureMainData(which='IF',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)
TF_Close = FutureMainData(which='TF',what='close',startDate=startDate,
                          endDate=Date.today().toString('%Y%m%d'),return_Data=True)

In [5]:
price_new = DataToDf(asset_price={'AL':AL_Close,'AU':AU_Close,'CU':CU_Close,
                         'B':B_Close,'PTA':PTA_Close,'CBOT':CBOT_Close,
                         'P':P_Close,'RB':RB_Close,'IF':IF_Close,'TF':TF_Close},fillzero=True)
price_new = (price_new * price.iloc[-1,]/1000).iloc[1:,:] #新数据进行价格处理

In [7]:
price = pd.concat([price,price_new],axis=0) #与本地数据进行合并

In [9]:
price.iloc[:-1,:].to_excel('price.xlsx') #更新本地数据

In [10]:
if sum(price.iloc[-1,].isna())==price.shape[1]:
    print('暂未更新前一交易日收盘价!')
elif (Date.today().weekday() == 5) or (Date.today().weekday() == 6):
    print(price.iloc[-5:,:])
else:
    print(price.iloc[-6:-1,:])

                    AL           AU           CU           B         PTA  \
Date                                                                       
2021-06-30  878.044930  1409.401613  5788.993719  624.636815  497.541264   
2021-07-01  869.404595  1425.815868  5777.989637  625.317247  496.168466   
2021-07-02  881.781292  1429.114060  5777.989637  637.020682  500.875202   
2021-07-05  899.295486  1433.869592  5884.644588  637.020682  519.309920   
2021-07-06  893.457421  1447.599273  5933.739724  637.020682  527.350595   

                   CBOT           P           RB           IF           TF  
Date                                                                        
2021-06-30  1969.652913  588.761784  1574.907249  2058.549160  1070.384147  
2021-07-01  1971.773480  609.120075  1574.601264  2043.880917  1069.422004  
2021-07-02  1976.368041  619.543520  1567.869583  2000.114052  1069.849623  
2021-07-05  1976.368041  619.869253  1597.856160  2006.774227  1069.956528  
2021-

计算均线

In [11]:
# 正常的ma
def create_data(sss,para,i=0):
    if sss.shape[1] == 1:
        train = sss.copy() #直接返回series
    else:
        train = sss.iloc[:,i:(i+1)].copy() #可以用dataframe调用
    train.columns = ['main']
    for i in para:
        train[('ma'+str(i))] = train['main'].rolling(i).mean()
    return train

# 指数的ma
def create_data_e(sss,para,i=0):
    if sss.shape[1] == 1:
        train = sss.copy()
    else:
        train = sss.iloc[:,i:(i+1)].copy()  
    train.columns = ['main']
    for i in para:
        train[('ma'+str(i))] = train['main'].ewm(span=i).mean()
    return train

发出信号函数

In [12]:
def signal(diff):
    '''敏感参数不涉及钝化操作，只作为逃顶逃底的判断'''
    diff[diff<0] = -1
    diff[diff>=0] = 1
    return diff #转化成多空信号

def signal_s(diff,delta,long):
    '''
    用来将双均线转化为逻辑信号
    -1代表开空平多，1代表开多平空，0代表多空均不持仓
    '''
    diff[abs(diff/te_train['ma'+str(long)])<delta] = 0 #设定空仓范围，diff<Δ时不进行任何持仓
    #钝化进场的时点
    diff[diff<0] = -1
    diff[diff>0] = 1
    return diff

In [13]:
def trend_follow_n_sig(train,diff):
    '''
    多空组合策略的方向信号;train是指训练数据;diff 表示普通参数信号差
    '''
    sig = np.array([0,0]) #初始化仓位信号
    diff = signal(diff) #将双均线差转化为-1 1信号
    
    for i in range(2,len(train)):  # 发出多开平空的信号
        if (diff[i-2] in [0,-1]) and (diff[i-1] == 1): #说明这是开多头平空头的时点
            sig = np.append(sig,1) #多头标志
        elif (diff[i-2] == 1) and (diff[i-1] == -1): #说明这是开空头平多头的时点
            sig = np.append(sig,-1) #多头标志        
        elif np.isnan(diff[i-2]) or np.isnan(diff[i-1]): #处理空值
            sig = np.append(sig,0) #因为在期初，同时双均线没有发出信号，所以信号为0
        elif (diff[i-2] == diff[i-1]):  # 说明信号与之前的保持不变
            sig = np.append(sig,sig[-1])
            
    return sig

def trend_follow_l_sig(train,diff):
    '''
    单边多头策略;train是指训练数据;diff 表示普通参数信号差
    '''
    sig = np.array([0, 0])
    diff = signal(diff) #将双均线差转化为 1 -1 信号
    
    for i in range(2,len(train)):  # 发出多开的信号
        if (diff[i-2] in [0,-1]) and (diff[i-1] == 1): #说明这是开多头的时点
            sig = np.append(sig, 1)
        elif (diff[i-2] == 1) and (diff[i-1] == -1): #说明这是平多头的时点
            sig = np.append(sig, 0)        
        elif np.isnan(diff[i-2]) or np.isnan(diff[i-1]): #处理空值
            sig = np.append(sig, 0)                
        elif (diff[i-2] == diff[i-1]):  # 说明说明信号与之前的保持不变
            sig = np.append(sig, sig[-1])
            
    return sig

def trend_follow_k_delta_sig(train,diff,diff1,kl,ks,delta): 
    '''
    多空组合增强策略;train是指训练数据
    diff 表示普通参数信号差;diff1表示敏感参数信号差
    k：表示逃顶参数;Δ表示钝化参数
    '''
    vol_l = (train['main'] / train['main'].rolling(50).min() - 1) #计算当日与50日内最低值的涨幅
    vol_s = (train['main'] / train['main'].rolling(50).max() - 1) #计算当日与50日内最大值的跌幅
    vol_l.fillna(0,inplace=True)
    vol_s.fillna(0,inplace=True) 

    sig = np.array([0, 0]) #信号初始化
    lng = 0  # 多头头寸
    sht = 0  # 空头头寸
    origin = 0 #空头开仓时初始价值(为了计算之后的策略净值变化)
    diff = signal_s(diff,delta,long) #将双均线差转化为-1 0 1信号
    diff1 = signal(diff1) #将敏感双均线差转化为-1 1信号
    
    for i in range(2,len(train)):  # 发出多开平空的信号
        if (diff[i-2] in [-1,1]) and (diff[i-1] == 0): #说明今天发出了平(多空)仓信号
            sig = np.append(sig, 0) #因为发出了平仓信号，所以信号记录应为0
            sht, lng = 0, 0 #处于空仓状态，所以多空方都没有持有头寸
        elif (diff[i-2] in [0,-1]) and (diff[i-1] == 1): #说明这是开多头的时点
            sig = np.append(sig, 1) #因为发出了多头信号，所以信号记录应该为1
            sht, lng = 0, 1 #多头方开仓
        elif (diff[i-2] in [0,1]) and (diff[i-1] == -1): #说明这是开空头的时点
            sig = np.append(sig, -1) #因为发出了空头信号，所以信号记录应该为-1
            lng, sht = 0, 1 #空头方开仓
        elif np.isnan(diff[i-2]) or np.isnan(diff[i-1]): #处理空值
            sig = np.append(sig, 0) #处理na的情况                
        elif (diff[i-2] == diff[i-1]):  # 说明记录持仓资产价格+现金即可
            if (vol_l[i-1] < kl) and (vol_s[i-1] > ks): #没有达到阈值
                if lng + sht !=0:
                    sig = np.append(sig, sig[-1]) #与上一个交易日保持一致
                elif lng + sht == 0:
                    if diff[i-1] == 1:
                        sig = np.append(sig, 1) #这里进行多头补仓
                        lng = 1
                    elif diff[i-1] == -1:
                        sig = np.append(sig, -1) #这里进行空头补仓
                        sht = 1
                    else:
                        sig = np.append(sig, 0) #这时应该是空仓状态            
            elif (vol_l[i-1] >= kl) or (vol_s[i-1] <= ks): #触发敏感系统
                if (diff1[i-2] * diff1[i-1]) == -1: #说明是逃顶或者逃底信号
                    if diff1[i-1] != diff[i-1]: #只有当两套系统方向相反时才会发出平仓信号
                        sig = np.append(sig, 0) #这里逃顶或者逃底都应该是空仓
                        sht, lng = 0, 0 #处于空仓状态，所以多空方都没有持有头寸
                    else: #否则应该保持之前的信号
                        sig = np.append(sig, sig[-1])
                elif (diff1[i-2] == diff1[i-1]):
                    if lng + sht !=0:
                        sig = np.append(sig, sig[-1]) #这里没有发出更多信号，应该和之前保持一致
                    elif lng + sht == 0: #规避掉那种虚假的回调
                        if diff1[i-1] == 1 and diff[i-1] == 1: #同时多头，那么就多头
                            sig = np.append(sig, 1)
                            lng = 1
                        elif diff1[i-1] == -1 and diff[i-1] == -1: #同时空头，那么就空头
                            sig = np.append(sig, -1)
                            sht = 1
                        else: #否则就啥都别干
                            sig = np.append(sig, 0) #保持空仓

    return sig

In [14]:
#计算调仓时点的函数
def sig_date(sig):
    for i in range(1, len(sig)):
        if sig[i]!=sig[i-1]:
            yield sig.index[i]

AL-铝

In [65]:
data = price.loc[:,['AL']].copy()
AL = data.copy()
data.columns = ['main']
para = np.array([2,60,1,30]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.018进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma2'] - te_train.loc[:,'ma60'], te_train.loc[:,'ma1'] - te_train.loc[:,'ma30']
long = 60
sig_AL = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_AL = pd.Series(sig_AL,index=data.index)
sig_AL.name = 'AL'
print('AL信号计算完成！')

AL信号计算完成！


In [69]:
# Date_sig_AL = pd.DatetimeIndex(np.array(list(sig_date(sig_AL))))
# Data_sig_AL = Data(Date_sig_AL,list(sig_AL.loc[Date_sig_AL].values),title='AL信号')
# Data_AL = Data(AL.index,list(AL.loc[:,'AL'].values),title='AL价格')
# Data_AL_2 = Data(AL.index,list(te_train.loc[:,'ma2'].values),title='AL2')
# Data_AL_60 = Data(AL.index,list(te_train.loc[:,'ma60'].values),title='AL60')
# Data_AL_1 = Data(AL.index,list(te_train.loc[:,'ma1'].values),title='AL1')
# Data_AL_30 = Data(AL.index,list(te_train.loc[:,'ma30'].values),title='AL30')
# plot(Graph(Data_sig_AL,fill='tozeroy',color='gray'),
#      Graph(Data_AL,yaxis='y2',width=2,color='red'),
#      Graph(Data_AL_2,yaxis='y2'),
#      Graph(Data_AL_60,yaxis='y2'),
#      Graph(Data_AL_1,yaxis='y2'),
#      Graph(Data_AL_30,yaxis='y2'))

AU-金

In [17]:
data = price.loc[:,['AU']].copy()
AU = data.copy()
data.columns = ['main']
para = np.array([8,15,4,7]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.01进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma8'] - te_train.loc[:,'ma15'], te_train.loc[:,'ma4'] - te_train.loc[:,'ma7']
long = 15
sig_AU = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_AU = pd.Series(sig_AU,index=data.index)
sig_AU.name = 'AU'
print('AU信号计算完成！')

AU信号计算完成！


In [66]:
# Date_sig_AU = pd.DatetimeIndex(np.array(list(sig_date(sig_AU))))
# Data_sig_AU = Data(Date_sig_AU,list(sig_AU.loc[Date_sig_AU].values),title='AU信号')
# Data_AU = Data(AU[AU.AU!=0].index,list(AU.loc[AU[AU.AU!=0].index,'AU'].values),title='AU价格')
# Data_AU_8 = Data(AU[AU.AU!=0].index,list(te_train.loc[AU[AU.AU!=0].index,'ma8'].values),title='AU8')
# Data_AU_15 = Data(AU[AU.AU!=0].index,list(te_train.loc[AU[AU.AU!=0].index,'ma15'].values),title='AU15')
# Data_AU_4 = Data(AU[AU.AU!=0].index,list(te_train.loc[AU[AU.AU!=0].index,'ma4'].values),title='AU4')
# Data_AU_7 = Data(AU[AU.AU!=0].index,list(te_train.loc[AU[AU.AU!=0].index,'ma7'].values),title='AU7')
# plot(Graph(Data_sig_AU,fill='tozeroy',color='gray'),
#      Graph(Data_AU,yaxis='y2',width=2,color='red'),
#      Graph(Data_AU_8,yaxis='y2'),
#      Graph(Data_AU_15,yaxis='y2'),
#      Graph(Data_AU_4,yaxis='y2'),
#      Graph(Data_AU_7,yaxis='y2'))

CU-铜

In [18]:
data = price.loc[:,['CU']].copy()
CU = data.copy()
data.columns = ['main']
para = np.array([4, 30]) #模型参数

test = data.loc[:,:] #测试集
te_train = create_data_e(test, para, 0)
diff = te_train.loc[:,'ma4'] - te_train.loc[:,'ma30']
sig_CU = trend_follow_n_sig(te_train, diff)
sig_CU = pd.Series(sig_CU,index=data.index)
sig_CU.name = 'CU'
print('CU信号计算完成！')

CU信号计算完成！


In [423]:
# Date_sig_CU = pd.DatetimeIndex(np.array(list(sig_date(sig_CU))))
# Data_sig_CU = Data(Date_sig_CU,list(sig_CU.loc[Date_sig_CU].values),title='CU信号')
# Data_CU = Data(CU[CU.CU!=0].index,list(CU.loc[CU[CU.CU!=0].index,'CU'].values),title='CU价格')
# Data_CU_4 = Data(CU[CU.CU!=0].index,list(te_train.loc[CU[CU.CU!=0].index,'ma4'].values),title='CU4')
# Data_CU_30 = Data(CU[CU.CU!=0].index,list(te_train.loc[CU[CU.CU!=0].index,'ma30'].values),title='CU30')
# plot(Graph(Data_sig_CU,fill='tozeroy',color='gray'),
#      Graph(Data_CU,yaxis='y2',width=2,color='red'),
#      Graph(Data_CU_4,yaxis='y2'),
#      Graph(Data_CU_30,yaxis='y2'))

SC-原油(B)

In [19]:
data = price.loc[:,['B']].copy()
B = data.copy()
data.columns = ['main']
para = np.array([4, 10]) #模型参数

test = data.loc[:,:] #测试集
te_train = create_data_e(test, para, 0)
diff = te_train.loc[:,'ma4'] - te_train.loc[:,'ma10']
sig_B = trend_follow_n_sig(te_train, diff)
sig_B = pd.Series(sig_B,index=data.index)
sig_B.name = 'B'
print('SC信号计算完成！')

SC信号计算完成！


In [424]:
# Date_sig_B = pd.DatetimeIndex(np.array(list(sig_date(sig_B))))
# Data_sig_B = Data(Date_sig_B,list(sig_B.loc[Date_sig_B].values),title='B信号')
# Data_B = Data(B[B.B!=0].index,list(B.loc[B[B.B!=0].index,'B'].values),title='B价格')
# Data_B_4 = Data(B[B.B!=0].index,list(te_train.loc[B[B.B!=0].index,'ma4'].values),title='B4')
# Data_B_10 = Data(B[B.B!=0].index,list(te_train.loc[B[B.B!=0].index,'ma10'].values),title='B10')
# plot(Graph(Data_sig_B,fill='tozeroy',color='gray'),
#      Graph(Data_B,yaxis='y2',width=2,color='red'),
#      Graph(Data_B_4,yaxis='y2'),
#      Graph(Data_B_10,yaxis='y2'))

PTA

In [20]:
data = price.loc[:,['PTA']].copy()
PTA = data.copy()
data.columns = ['main']
para = np.array([4,45,2,22]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.01进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma4'] - te_train.loc[:,'ma45'], te_train.loc[:,'ma2'] - te_train.loc[:,'ma22']
long = 45
sig_PTA = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_PTA = pd.Series(sig_PTA,index=data.index)
sig_PTA.name = 'PTA'
print('PTA信号计算完成！')

PTA信号计算完成！


In [661]:
# Date_sig_PTA = pd.DatetimeIndex(np.array(list(sig_date(sig_PTA))))
# Data_sig_PTA = Data(Date_sig_PTA,list(sig_PTA.loc[Date_sig_PTA].values),title='PTA信号')
# Data_PTA = Data(PTA[PTA.PTA!=0].index,list(PTA.loc[PTA[PTA.PTA!=0].index,'PTA'].values),title='PTA价格')
# Data_PTA_4 = Data(PTA[PTA.PTA!=0].index,list(te_train.loc[PTA[PTA.PTA!=0].index,'ma4'].values),title='PTA4')
# Data_PTA_45 = Data(PTA[PTA.PTA!=0].index,list(te_train.loc[PTA[PTA.PTA!=0].index,'ma45'].values),title='PTA45')
# Data_PTA_2 = Data(PTA[PTA.PTA!=0].index,list(te_train.loc[PTA[PTA.PTA!=0].index,'ma2'].values),title='PTA2')
# Data_PTA_22 = Data(PTA[PTA.PTA!=0].index,list(te_train.loc[PTA[PTA.PTA!=0].index,'ma22'].values),title='PTA22')
# plot(Graph(Data_sig_PTA,fill='tozeroy',color='gray'),
#      Graph(Data_PTA,yaxis='y2',width=2,color='red'),
#      Graph(Data_PTA_4,yaxis='y2'),
#      Graph(Data_PTA_45,yaxis='y2'),
#      Graph(Data_PTA_2,yaxis='y2'),
#      Graph(Data_PTA_22,yaxis='y2'))

CBOT-美豆

In [21]:
data = price.loc[:,['CBOT']].copy()
CBOT = data.copy()
data.columns = ['main']
para = np.array([8,25,4,12]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.01进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma8'] - te_train.loc[:,'ma25'], te_train.loc[:,'ma4'] - te_train.loc[:,'ma12']
long = 25
sig_CBOT = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_CBOT = pd.Series(sig_CBOT,index=data.index)
sig_CBOT.name = 'CBOT'
print('CBOT大豆信号计算完成！')

CBOT大豆信号计算完成！


In [664]:
# Date_sig_CBOT = pd.DatetimeIndex(np.array(list(sig_date(sig_CBOT))))
# Data_sig_CBOT = Data(Date_sig_CBOT,list(sig_CBOT.loc[Date_sig_CBOT].values),title='CBOT信号')
# Data_CBOT = Data(CBOT[CBOT.CBOT!=0].index,list(CBOT.loc[CBOT[CBOT.CBOT!=0].index,'CBOT'].values),title='CBOT价格')
# Data_CBOT_8 = Data(CBOT[CBOT.CBOT!=0].index,list(te_train.loc[CBOT[CBOT.CBOT!=0].index,'ma8'].values),title='CBOT8')
# Data_CBOT_25 = Data(CBOT[CBOT.CBOT!=0].index,list(te_train.loc[CBOT[CBOT.CBOT!=0].index,'ma25'].values),title='CBOT25')
# Data_CBOT_4 = Data(CBOT[CBOT.CBOT!=0].index,list(te_train.loc[CBOT[CBOT.CBOT!=0].index,'ma4'].values),title='CBOT4')
# Data_CBOT_12 = Data(CBOT[CBOT.CBOT!=0].index,list(te_train.loc[CBOT[CBOT.CBOT!=0].index,'ma12'].values),title='CBOT12')
# plot(Graph(Data_sig_CBOT,fill='tozeroy',color='gray'),
#      Graph(Data_CBOT,yaxis='y2',width=2,color='red'),
#      Graph(Data_CBOT_8,yaxis='y2'),
#      Graph(Data_CBOT_25,yaxis='y2'),
#      Graph(Data_CBOT_4,yaxis='y2'),
#      Graph(Data_CBOT_12,yaxis='y2'))

P-棕榈油

In [22]:
data = price.loc[:,['P']].copy()
P = data.copy()
data.columns = ['main']
para = np.array([10,45,5,22]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.01进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma10'] - te_train.loc[:,'ma45'], te_train.loc[:,'ma5'] - te_train.loc[:,'ma22']
long = 45
sig_P = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_P = pd.Series(sig_P,index=data.index)
sig_P.name = 'P'
print('P信号计算完成！')

P信号计算完成！


In [667]:
# Date_sig_P = pd.DatetimeIndex(np.array(list(sig_date(sig_P))))
# Data_sig_P = Data(Date_sig_P,list(sig_P.loc[Date_sig_P].values),title='P信号')
# Data_P = Data(P[P.P!=0].index,list(P.loc[P[P.P!=0].index,'P'].values),title='P价格')
# Data_P_10 = Data(P[P.P!=0].index,list(te_train.loc[P[P.P!=0].index,'ma10'].values),title='P10')
# Data_P_45 = Data(P[P.P!=0].index,list(te_train.loc[P[P.P!=0].index,'ma45'].values),title='P45')
# Data_P_5 = Data(P[P.P!=0].index,list(te_train.loc[P[P.P!=0].index,'ma5'].values),title='P5')
# Data_P_22 = Data(P[P.P!=0].index,list(te_train.loc[P[P.P!=0].index,'ma22'].values),title='P22')
# plot(Graph(Data_sig_P,fill='tozeroy',color='gray'),
#      Graph(Data_P,yaxis='y2',width=2,color='red'),
#      Graph(Data_P_10,yaxis='y2'),
#      Graph(Data_P_45,yaxis='y2'),
#      Graph(Data_P_5,yaxis='y2'),
#      Graph(Data_P_22,yaxis='y2'))

RB-螺纹钢

In [23]:
data = price.loc[:,['RB']].copy()
RB = data.copy()
data.columns = ['main']
para = np.array([4,30,2,15]) #模型参数
returns = data['main'] / data['main'].shift(1) - 1 #收益率
kl = np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.7 #上行敏感阈值
ks = - np.sqrt(252)*np.std(returns.loc['2017':'2021-6-5'])*0.4 #下行敏感阈值
cos = 0.2389198448279757 #基准参考波动率
delta = 0.018 * ((np.std((data/data.shift(1) - 1))*np.sqrt(252))['main']) / cos #0.01进行个性化调整钝化阈值

test = data.loc[:,:] #测试集
te_train = create_data(test, para, 0)
diff, diff1 = te_train.loc[:,'ma4'] - te_train.loc[:,'ma30'], te_train.loc[:,'ma2'] - te_train.loc[:,'ma15']
long = 30
sig_RB = trend_follow_k_delta_sig(te_train, diff, diff1, kl, ks, delta)
sig_RB = pd.Series(sig_RB,index=data.index)
sig_RB.name = 'RB'
print('RB信号计算完成！')

RB信号计算完成！


In [670]:
# Date_sig_RB = pd.DatetimeIndex(np.array(list(sig_date(sig_RB))))
# Data_sig_RB = Data(Date_sig_RB,list(sig_RB.loc[Date_sig_RB].values),title='RB信号')
# Data_RB = Data(RB[RB.RB!=0].index,list(RB.loc[RB[RB.RB!=0].index,'RB'].values),title='RB价格')
# Data_RB_4 = Data(RB[RB.RB!=0].index,list(te_train.loc[RB[RB.RB!=0].index,'ma4'].values),title='RB4')
# Data_RB_30 = Data(RB[RB.RB!=0].index,list(te_train.loc[RB[RB.RB!=0].index,'ma30'].values),title='RB30')
# Data_RB_2 = Data(RB[RB.RB!=0].index,list(te_train.loc[RB[RB.RB!=0].index,'ma2'].values),title='RB2')
# Data_RB_15 = Data(RB[RB.RB!=0].index,list(te_train.loc[RB[RB.RB!=0].index,'ma15'].values),title='RB15')
# plot(Graph(Data_sig_RB,fill='tozeroy',color='gray'),
#      Graph(Data_RB,yaxis='y2',width=2,color='red'),
#      Graph(Data_RB_4,yaxis='y2'),
#      Graph(Data_RB_30,yaxis='y2'),
#      Graph(Data_RB_2,yaxis='y2'),
#      Graph(Data_RB_15,yaxis='y2'))

IF-股指单边多头

In [24]:
data = price.loc[:,['IF']].copy()
IF = data.copy()
data.columns = ['main']
para = np.array([4, 45]) #模型参数

test = data.loc[:,:] #测试集
te_train = create_data_e(test, para, 0)
diff = te_train.loc[:,'ma4'] - te_train.loc[:,'ma45']
sig_IF = trend_follow_l_sig(te_train, diff)
sig_IF = pd.Series(sig_IF,index=data.index)
sig_IF.name = 'IF'
print('IF信号计算完成！')

IF信号计算完成！


In [429]:
# Date_sig_IF = pd.DatetimeIndex(np.array(list(sig_date(sig_IF))))
# Data_sig_IF = Data(Date_sig_IF,list(sig_IF.loc[Date_sig_IF].values),title='IF信号')
# Data_IF = Data(IF[IF.IF!=0].index,list(IF.loc[IF[IF.IF!=0].index,'IF'].values),title='IF价格')
# Data_IF_4 = Data(IF[IF.IF!=0].index,list(te_train.loc[IF[IF.IF!=0].index,'ma4'].values),title='IF4')
# Data_IF_45 = Data(IF[IF.IF!=0].index,list(te_train.loc[IF[IF.IF!=0].index,'ma45'].values),title='IF45')
# plot(Graph(Data_sig_IF,fill='tozeroy',color='gray'),
#      Graph(Data_IF,yaxis='y2',width=2,color='red'),
#      Graph(Data_IF_4,yaxis='y2'),
#      Graph(Data_IF_45,yaxis='y2'))

TF-国债单边多头

In [25]:
data = price.loc[:,['TF']].copy()
TF = data.copy()
data.columns = ['main']
para = np.array([8, 45]) #模型参数

test = data.loc[:,:] #测试集
te_train = create_data_e(test, para, 0)
diff = te_train.loc[:,'ma8'] - te_train.loc[:,'ma45']
sig_TF = trend_follow_l_sig(te_train, diff)
sig_TF = pd.Series(sig_TF,index=data.index)
sig_TF.name = 'TF'
print('TF信号计算完成！')

TF信号计算完成！


In [430]:
# Date_sig_TF = pd.DatetimeIndex(np.array(list(sig_date(sig_TF))))
# Data_sig_TF = Data(Date_sig_TF,list(sig_TF.loc[Date_sig_TF].values),title='TF信号')
# Data_TF = Data(TF[TF.TF!=0].index,list(TF.loc[TF[TF.TF!=0].index,'TF'].values),title='TF价格')
# Data_TF_8 = Data(TF[TF.TF!=0].index,list(te_train.loc[TF[TF.TF!=0].index,'ma8'].values),title='TF8')
# Data_TF_45 = Data(TF[TF.TF!=0].index,list(te_train.loc[TF[TF.TF!=0].index,'ma45'].values),title='TF45')
# plot(Graph(Data_sig_TF,fill='tozeroy',color='gray'),
#      Graph(Data_TF,yaxis='y2',width=2,color='red'),
#      Graph(Data_TF_8,yaxis='y2'),
#      Graph(Data_TF_45,yaxis='y2'))

In [26]:
print('所有单品种信号计算完成！')

所有单品种信号计算完成！


将多空方向汇总

In [27]:
sig = pd.concat([sig_AL,sig_AU,sig_CU,
                 sig_B,sig_PTA,sig_CBOT,
                 sig_P,sig_RB,sig_IF,sig_TF],axis=1) #构建单品种策略的信号DataFrame

In [28]:
print('最近5个交易日各品种信号:\n')
sig.tail(5)

最近5个交易日各品种信号:



,AL,AU,CU,B,PTA,CBOT,P,RB,IF,TF
Date,,,,,,,,,,
2021-07-01,1,-1,-1,1,1,0,-1,1,1,1
2021-07-02,0,-1,-1,1,1,0,-1,1,1,1
2021-07-05,0,-1,-1,1,1,0,-1,1,0,1
2021-07-06,1,-1,-1,1,1,0,0,1,0,1
2021-07-07,1,-1,-1,1,1,0,0,1,0,1


In [29]:
def sig_date(data): #返回给定资产的信号发出日期
    for i in range(60,len(data)):
        if (i==60) and (data.iloc[i]==0):
            continue #如果本资产累计60日数据但是信号为空仓，则暂时不作为调仓时点纳入
        if (i==60) and (data.iloc[i]!=0):
            yield data.index[i] #反之，如果本资产累计60日数据但是信号为操作，则作为调仓时点纳入
        if data.iloc[i] != data.iloc[i-1]:
            yield data.index[i] #其他情况下，如果前后两个交易日信号不同，代表进行交易，记录日期

In [30]:
a = map(sig_date, [sig_AL,sig_AU,sig_CU,
                   sig_B,sig_PTA,sig_CBOT,
                   sig_P,sig_RB,sig_IF,sig_TF])
X = list(a)
y = set()
for i in range(len(X)):
    x = set(X[i])
    y = y.union(x)

y=list(y)
signals = np.sort(np.array(y)) #获取调仓时点

ret = (price - price.shift(1)) / price.shift(1)
ret[ret == np.inf] = np.nan #计算收益率矩阵

策略目标波动率：20%

In [31]:
#直接使用固定权重进行加减仓位,截止到2021.7.1
cov = opt.risk_models.CovarianceShrinkage(ret.loc[:'2021-7-1',],returns_data=True).ledoit_wolf() #样本协方差估计
mu = opt.expected_returns.mean_historical_return(ret.loc[:'2021-7-1',],returns_data=True) #样本均值估计
ef = opt.EfficientFrontier(mu, cov) #初始化类
ef.add_objective(opt.objective_functions.L2_reg, gamma=1)
weights = ef.efficient_risk(0.2)#采用最小方差估计
w = dict(ef.clean_weights()) #给出最终的分配权重
w = np.array([w.get(i,0) for i in price.columns]) * 0.2 / ef.portfolio_performance()[1]

In [34]:
print('最近两个交易日调仓结果：\n')
balance_result = (sig.loc[signals,]*w).iloc[-2:,]
balance_result['record'] = np.array([['上次调仓'],['本次调仓']])
balance_result.index.name='调仓日期'
balance_result

最近两个交易日调仓结果：



,AL,AU,CU,B,PTA,CBOT,P,RB,IF,TF,record
调仓日期,,,,,,,,,,,
2021-07-05,0.000000,-0.229611,-0.315456,0.05547,0.149548,0.0,-0.162012,0.241614,0.0,0.210004,上次调仓
2021-07-06,0.191967,-0.229611,-0.315456,0.05547,0.149548,0.0,0.000000,0.241614,0.0,0.210004,本次调仓


In [33]:
# FW = sig.loc[price.index[0:1],].copy().fillna(0) #初始化权重矩阵
# for j in range(len(signals)):
#     start = np.where(price.index==signals[j])[0][0] #记录当前调仓时点
#     na = (price.loc[price.index[start-60:start],] == 0).apply(sum,axis=0) #过去60日有数据?
#     pooling = na[na<=5].index.values.tolist() #获取能够组合的品种代码
    
#     rets = ret.loc[price.index[(start-60):start],pooling].copy() #记录能够组合的品种的过去60日收益率
#     trans = sig.loc[price.index[start],pooling] #记录当前的调仓操作方向
#     rets.loc[:,trans[trans==-1].index] = - rets.loc[:,trans[trans==-1].index] #将空头方向的收益率反转
    
#     cov = opt.risk_models.CovarianceShrinkage(rets,returns_data=True).ledoit_wolf() #样本协方差估计
#     mu = opt.expected_returns.mean_historical_return(rets,returns_data=True) #样本均值估计
#     ef = opt.EfficientFrontier(mu, cov) #初始化类
#     weights = ef.min_volatility() #采用最小方差估计
#     w = dict(ef.clean_weights()) #给出最终的分配权重
#     w = np.array([w.get(i,0) for i in price.columns]) * 0.2 / ef.portfolio_performance()[1] #伸缩为相应的目标波动率:20%
    
#     if start != (price.shape[0] - 1):
#         for i in range(len(price.columns)): #用来判断当调仓日需要改变的股票能不能调整
#             if price.iloc[start, i] == price.iloc[start-1, i]:
#                 w[i] = FW.iloc[-1, i]

#     fw = sig.loc[price.index[start:start+1],] * w #空头方向的头寸应该为负数
#     FW = FW.append(fw, ignore_index=False) #添加到调仓序列中

In [112]:
# weights = FW.iloc[1:,] #去除权重初始化行

In [113]:
# balance_result = weights.iloc[-2:,]
# balance_result['record'] = np.array([['上次调仓'],['本次调仓']])
# balance_result.index.name='调仓日期'
# print('最近两个交易日调仓结果：\n')
# balance_result #显实最近两次调仓的结果

In [35]:
AA = Asset_Allocation(weights=sig.loc[signals,]*w,asset_price={'AL':Data(price.index,list(price['AL'].values),title='AL'),
                                                   'AU':Data(price.index,list(price['AU'].values),title='AU'),
                                                   'CU':Data(price.index,list(price['CU'].values),title='CU'),
                                                   'B':Data(price.index,list(price['B'].values),title='B'),
                                                   'PTA':Data(price.index,list(price['PTA'].values),title='PTA'),
                                                   'CBOT':Data(price.index,list(price['CBOT'].values),title='CBOT'),
                                                   'P':Data(price.index,list(price['P'].values),title='P'),
                                                   'RB':Data(price.index,list(price['RB'].values),title='RB'),
                                                   'IF':Data(price.index,list(price['IF'].values),title='IF'),
                                                   'TF':Data(price.index,list(price['TF'].values),title='TF')},
                      notice=False,rebalance_delay=1) #控制波动率20%，加减仓正常，同时交易信号发出后一个交易日调仓

与其他策略对比(时间区间：2020-5-22-最新，将2021-6-4设为净值1)

In [36]:
NAV = AA.asset_nav.iloc[:,-1]
new = NAV.loc['2020-5-22':] / NAV.loc['2021-6-4']
strategy = Data(new.index,list(new.values),title='策略样本外净值')

# fof1 = ProdNetData(prodCode='AA1')
hs300 = edbData(seriesCode='M0020209', startingDate=Date.fromString(date='20200522',format='%Y%m%d'), title='沪深300指数')
# fof1 = DataToDf({'FOF1号':fof1.data})
# fof1 = fof1.loc['2020-5-22':,:]
# fof1.loc[:,'FOF1号'] = fof1.loc[:,'FOF1号'] / fof1.loc['2021-6-4','FOF1号']
hs300 = DataToDf({'沪深300':hs300})
hs300.loc[:,'沪深300'] = hs300.loc[:,'沪深300'] / hs300.loc['2021-6-4','沪深300']

hs = Data(hs300.index,list(hs300['沪深300'].values),title='沪深300')
# fof = Data(fof1.index,list(fof1['FOF1号'].values),title='FOF1号')

In [37]:
# ret_mean_f = np.mean(np.array(fof.momPercentage(1).dataList))*(52)
# vol_f = np.std(np.array(fof.momPercentage(1).dataList))*np.sqrt(52)
# sharpe_f = round(ret_mean_f / vol_f,3)
# drawdown_f = max(1- pd.Series(fof.dataList) / pd.Series(fof.dataList).cummax())
ret_mean_s = np.mean(np.array(strategy.momPercentage(1).dataList))*(252)
vol_s = np.std(np.array(strategy.momPercentage(1).dataList))*np.sqrt(252)
sharpe_s = round(ret_mean_s / vol_s,3)
drawdown_s = max(1- pd.Series(strategy.dataList) / pd.Series(strategy.dataList).cummax())
ret_mean_h = np.mean(np.array(hs.momPercentage(1).dataList))*(252)
vol_h = np.std(np.array(hs.momPercentage(1).dataList))*np.sqrt(252)
sharpe_h = round(ret_mean_h / vol_h,3)
drawdown_h = max(1- pd.Series(hs.dataList) / pd.Series(hs.dataList).cummax())

In [38]:
dat = pd.DataFrame({'稳健趋势追随策略':np.array([str(round(100*ret_mean_s,2))+'%',str(round(100*vol_s,2))+'%',
                                         sharpe_s,str(round(100*drawdown_s,2))+'%']),
                    '沪深300':np.array([str(round(100*ret_mean_h,2))+'%',str(round(100*vol_h,2))+'%',
                                      sharpe_h,str(round(100*drawdown_h,2))+'%'])},
                  index=['年化收益','年化波动','夏普比率','最大回撤'])

In [118]:
# dat = pd.DataFrame({'FOF':np.array([str(round(100*ret_mean_f,2))+'%',str(round(100*vol_f,2))+'%',
#                                     sharpe_f,str(round(100*drawdown_f,2))+'%']),
#                     '稳健趋势追随策略':np.array([str(round(100*ret_mean_s,2))+'%',str(round(100*vol_s,2))+'%',
#                                          sharpe_s,str(round(100*drawdown_s,2))+'%']),
#                     '沪深300':np.array([str(round(100*ret_mean_h,2))+'%',str(round(100*vol_h,2))+'%',
#                                       sharpe_h,str(round(100*drawdown_h,2))+'%'])},
#                   index=['年化收益','年化波动','夏普比率','最大回撤'])

In [39]:
print('策略表现综述:\n')
dat

策略表现综述:



,稳健趋势追随策略,沪深300
年化收益,38.8%,28.41%
年化波动,19.08%,20.59%
夏普比率,2.034,1.38
最大回撤,18.75%,15.18%


In [40]:
drawdown = pd.Series(strategy.dataList) / pd.Series(strategy.dataList).cummax() - 1
drawdown = Data(new.index,list(drawdown.values),title='策略回撤')

In [41]:
plot(Graph(drawdown,fill='tozeroy',color='gray'),Graph(hs,yaxis='y2',width=2),Graph(strategy,yaxis='y2',width=2))

In [48]:
# plot(Graph(drawdown,fill='tozeroy',color='gray'),Graph(hs,yaxis='y2',width=2),
#      Graph(fof,yaxis='y2',width=2),Graph(strategy,yaxis='y2',width=2))

单品种择时效果展示

In [42]:
single_weights = (sig.loc[signals,]*w / abs(sig.loc[signals,]*w)).fillna(0)

AL CU对比图

In [43]:
AL = Asset_Allocation(weights=single_weights.loc[:,['AL']],asset_price={'AL':Data(price.index,list(price['AL'].values),title='AL')},
                notice=False,rebalance_delay=1)
AL_val = AL.asset_nav.loc['2020-5-22':,'组合净值'] / AL.asset_nav.loc['2021-6-4','组合净值']
AL_val = Data(AL_val.index,list(AL_val.values),title='AL:策略净值')
AL_sig = (AL.asset_shares / abs(AL.asset_shares)).fillna(0)
AL_sig = Data(AL_sig.loc['2020-5-22':,'AL'].index,list(AL_sig.loc['2020-5-22':,'AL'].values),title='AL:择时信号')

CU = Asset_Allocation(weights=single_weights.loc[:,['CU']],asset_price={'CU':Data(price.index,list(price['CU'].values),title='CU')},
                notice=False,rebalance_delay=1)
CU_val = CU.asset_nav.loc['2020-5-22':,'组合净值'] / CU.asset_nav.loc['2021-6-4','组合净值']
CU_val = Data(CU_val.index,list(CU_val.values),title='CU:策略净值')
CU_sig = (CU.asset_shares / abs(CU.asset_shares)).fillna(0)
CU_sig = Data(CU_sig.loc['2020-5-22':,'CU'].index,list(CU_sig.loc['2020-5-22':,'CU'].values),title='CU:择时信号')

In [44]:
doubleplot([Graph(AL_sig,fill='tozeroy',color='gray'),Graph(AL_val,yaxis='y2')],
           [Graph(CU_sig,fill='tozeroy',color='gray'),Graph(CU_val,yaxis='y2')],y4tickformat='.2f',fontsize=10)

IF TF对比图

In [45]:
IF = Asset_Allocation(weights=single_weights.loc[:,['IF']],asset_price={'IF':Data(price.index,list(price['IF'].values),title='IF')},
                notice=False,rebalance_delay=1)
IF_val = IF.asset_nav.loc['2020-5-22':,'组合净值'] / IF.asset_nav.loc['2021-6-4','组合净值']
IF_val = Data(IF_val.index,list(IF_val.values),title='IF:策略净值')
IF_sig = (IF.asset_shares / abs(IF.asset_shares)).fillna(0)
IF_sig = Data(IF_sig.loc['2020-5-22':,'IF'].index,list(IF_sig.loc['2020-5-22':,'IF'].values),title='IF:择时信号')

TF = Asset_Allocation(weights=single_weights.loc[:,['TF']],asset_price={'TF':Data(price.index,list(price['TF'].values),title='TF')},
                notice=False,rebalance_delay=1)
TF_val = TF.asset_nav.loc['2020-5-22':,'组合净值'] / TF.asset_nav.loc['2021-6-4','组合净值']
TF_val = Data(TF_val.index,list(TF_val.values),title='TF:策略净值')
TF_sig = (TF.asset_shares / abs(TF.asset_shares)).fillna(0)
TF_sig = Data(TF_sig.loc['2020-5-22':,'TF'].index,list(TF_sig.loc['2020-5-22':,'TF'].values),title='TF:择时信号')

In [46]:
doubleplot([Graph(IF_sig,fill='tozeroy',color='gray'),Graph(IF_val,yaxis='y2')],
           [Graph(TF_sig,fill='tozeroy',color='gray'),Graph(TF_val,yaxis='y2')],y2tickformat='.3f',y4tickformat='.5f',fontsize=10)

CBOT P 对比图

In [47]:
CBOT = Asset_Allocation(weights=single_weights.loc[:,['CBOT']],asset_price={'CBOT':Data(price.index,list(price['CBOT'].values),title='CBOT')},
                notice=False,rebalance_delay=1)
CBOT_val = CBOT.asset_nav.loc['2020-5-22':,'组合净值'] / CBOT.asset_nav.loc['2021-6-4','组合净值']
CBOT_val = Data(CBOT_val.index,list(CBOT_val.values),title='CBOT:策略净值')
CBOT_sig = (CBOT.asset_shares / abs(CBOT.asset_shares)).fillna(0)
CBOT_sig = Data(CBOT_sig.loc['2020-5-22':,'CBOT'].index,list(CBOT_sig.loc['2020-5-22':,'CBOT'].values),title='CBOT:择时信号')

P = Asset_Allocation(weights=single_weights.loc[:,['P']],asset_price={'P':Data(price.index,list(price['P'].values),title='P')},
                notice=False,rebalance_delay=1)
P_val = P.asset_nav.loc['2020-5-22':,'组合净值'] / P.asset_nav.loc['2021-6-4','组合净值']
P_val = Data(P_val.index,list(P_val.values),title='P:策略净值')
P_sig = (P.asset_shares / abs(P.asset_shares)).fillna(0)
P_sig = Data(P_sig.loc['2020-5-22':,'P'].index,list(P_sig.loc['2020-5-22':,'P'].values),title='P:择时信号')

In [48]:
doubleplot([Graph(CBOT_sig,fill='tozeroy',color='gray'),Graph(CBOT_val,yaxis='y2')],
           [Graph(P_sig,fill='tozeroy',color='gray'),Graph(P_val,yaxis='y2')],y4tickformat='.2f',
           y2tickformat='.2f',fontsize=10)

PTA B 对比图

In [49]:
PTA = Asset_Allocation(weights=single_weights.loc[:,['PTA']],asset_price={'PTA':Data(price.index,list(price['PTA'].values),title='PTA')},
                notice=False,rebalance_delay=1)
PTA_val = PTA.asset_nav.loc['2020-5-22':,'组合净值'] / PTA.asset_nav.loc['2021-6-4','组合净值']
PTA_val = Data(PTA_val.index,list(PTA_val.values),title='PTA:策略净值')
PTA_sig = (PTA.asset_shares / abs(PTA.asset_shares)).fillna(0)
PTA_sig = Data(PTA_sig.loc['2020-5-22':,'PTA'].index,list(PTA_sig.loc['2020-5-22':,'PTA'].values),title='PTA:择时信号')

B = Asset_Allocation(weights=single_weights.loc[:,['B']],asset_price={'B':Data(price.index,list(price['B'].values),title='B')},
                notice=False,rebalance_delay=1)
B_val = B.asset_nav.loc['2020-5-22':,'组合净值'] / B.asset_nav.loc['2021-6-4','组合净值']
B_val = Data(B_val.index,list(B_val.values),title='B:策略净值')
B_sig = (B.asset_shares / abs(B.asset_shares)).fillna(0)
B_sig = Data(B_sig.loc['2020-5-22':,'B'].index,list(B_sig.loc['2020-5-22':,'B'].values),title='B:择时信号')

In [50]:
doubleplot([Graph(PTA_sig,fill='tozeroy',color='gray'),Graph(PTA_val,yaxis='y2')],
           [Graph(B_sig,fill='tozeroy',color='gray'),Graph(B_val,yaxis='y2')],y4tickformat='.2f',
           y2tickformat='.2f',fontsize=10)

AU RB 对比图

In [51]:
AU = Asset_Allocation(weights=single_weights.loc[:,['AU']],asset_price={'AU':Data(price.index,list(price['AU'].values),title='AU')},
                notice=False,rebalance_delay=1)
AU_val = AU.asset_nav.loc['2020-5-22':,'组合净值'] / AU.asset_nav.loc['2021-6-4','组合净值']
AU_val = Data(AU_val.index,list(AU_val.values),title='AU:策略净值')
AU_sig = (AU.asset_shares / abs(AU.asset_shares)).fillna(0)
AU_sig = Data(AU_sig.loc['2020-5-22':,'AU'].index,list(AU_sig.loc['2020-5-22':,'AU'].values),title='AU:择时信号')

RB = Asset_Allocation(weights=single_weights.loc[:,['RB']],asset_price={'RB':Data(price.index,list(price['RB'].values),title='RB')},
                notice=False,rebalance_delay=1)
RB_val = RB.asset_nav.loc['2020-5-22':,'组合净值'] / RB.asset_nav.loc['2021-6-4','组合净值']
RB_val = Data(RB_val.index,list(RB_val.values),title='RB:策略净值')
RB_sig = (RB.asset_shares / abs(RB.asset_shares)).fillna(0)
RB_sig = Data(RB_sig.loc['2020-5-22':,'RB'].index,list(RB_sig.loc['2020-5-22':,'RB'].values),title='RB:择时信号')

In [52]:
doubleplot([Graph(AU_sig,fill='tozeroy',color='gray'),Graph(AU_val,yaxis='y2')],
           [Graph(RB_sig,fill='tozeroy',color='gray'),Graph(RB_val,yaxis='y2')],y4tickformat='.2f',
           y2tickformat='.2f',fontsize=10)

净值全景图

注：因为涉及样本内的净值走势，所以参考价值并不大，但是仍然将其展示出来以体会趋势的力量，并认识到当市场整体缺乏趋势之时，即使是在样本内，经过一定的挖掘，策略表现都会平平无奇

In [53]:
AA.plot_performance(yearLine=True) #全景净值走势

请选择您的无风险利率参考指标(输入SHIBOR3M代表SHIBOR3个月、或GKYLD1Y代表国开1年收益率、或者数值如0.03)SHIBOR3M
Rf长度-3685
